# EPC Analysis
key questions:
- what ratio of records don't have a UPRN
- how can this gap be filled
- what ratio is below E compliance

In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
output_file = os.getenv('EPC_OUTPUT_FILE')
all_epc = pd.read_csv(
    output_file, 
    index_col=0, 
    dtype={'transaction-type': str, 'aircon-present': str}, 
    parse_dates=['lodgement-date', 'lodgement-datetime']
)
# len(all_epc) # 1378713 before any filtering
# remove small properties
all_epc = all_epc[all_epc['floor-area'] >= 40] # m2
dedup = all_epc.sort_values('lodgement-datetime') .drop_duplicates(subset=['building-reference-number'], keep='last')
# df.sort_values('DATE_CHANGED').drop_duplicates('STATION_ID',keep='last')
len(dedup) # 1217573 (1281342 before filter by floor area)

1217573

In [51]:

dedup["uprn"].isna().sum() # 440164

440164

In [42]:
# some of the numbers are from before filter by size
len(dedup[(dedup["lodgement-date"] >= "2014-01-01") & (dedup["uprn"].isna())])  # 282031 - no uprn in the last 10 years - about 1/3

282031

In [36]:
dedup[
    # (dedup["lodgement-date"] >= "2014-01-01") & 
    (dedup["uprn"].isna())
 ].groupby(dedup["lodgement-date"].map(lambda x: x.year)).size()

lodgement-date
2008     9016
2009    38816
2010    24602
2011    26373
2012    36033
2013    28245
2014    25488
2015    25989
2016    24644
2017    27665
2018    33098
2019    32698
2020    21022
2021    23322
2022    31538
2023    40926
2024    13231
dtype: int64

In [43]:
dedup[
    # (dedup["lodgement-date"] >= "2014-01-01") &
    # (dedup["uprn"].isna()) &
    (dedup['asset-rating-band'].isin(['F', 'G']))
    ].groupby(dedup["lodgement-date"].map(lambda x: x.year)).size()

lodgement-date
2008     2400
2009    14680
2010    10458
2011    10706
2012    15649
2013    10740
2014     8716
2015     7866
2016     6828
2017     5253
2018     2848
2019     2136
2020     1383
2021     1322
2022     1297
2023     1175
2024      478
dtype: int64

In [49]:
dedup[
    # (dedup["lodgement-date"] >= "2014-01-01") &
    (dedup["uprn"].isna()) &
    (dedup['asset-rating-band'].isin(['F', 'G']))
][30000:30020]

,address1,address2,address3,postcode,building-reference-number,asset-rating,asset-rating-band,property-type,inspection-date,local-authority,...,ac-inspection-commissioned,building-environment,address,local-authority-label,constituency-label,posttown,lodgement-datetime,primary-energy-value,uprn,uprn-source
lmk-key,,,,,,,,,,,,,,,,,,,,,
98541380302015011616320806300990,NaN,Unit B,Peacock View,ST4 2XJ,435588300000,144,F,B1 Offices and Workshop businesses,2015-01-16,E06000021,...,4.0,Heating and Natural Ventilation,"Unit B, Peacock View",Stoke-on-Trent,Stoke-on-Trent Central,STOKE-ON-TRENT,2015-01-16 16:32:08,NaN,NaN,NaN
98534190502015011616491150100990,NaN,Unit 18,Whieldon Industrial Estate,ST4 4JP,255879150000,130,F,B1 Offices and Workshop businesses,2015-01-10,E06000021,...,4.0,Heating and Mechanical Ventilation,"Unit 18, Whieldon Industrial Estate",Stoke-on-Trent,Stoke-on-Trent South,STOKE-ON-TRENT,2015-01-16 16:49:11,NaN,NaN,NaN
98555568912015011618234108029816,GROUND FLOOR OFFICES,57-63 Church Road,NaN,SW19 5SB,869988010002,183,G,B1 Offices and Workshop businesses,2015-01-16,E09000024,...,5.0,Air Conditioning,"GROUND FLOOR OFFICES, 57-63 Church Road",Merton,Wimbledon,LONDON,2015-01-16 18:23:41,NaN,NaN,NaN
357bd4800943b435ba2a8f9a8f0a1bd8f3edb4a4d20e69dd22786e52457e59d1,NaN,Hose Village Hall,2 Harby Lane,LE14 4JR,10003531314,127,F,D1 Non-residential Institutions - Community/Da...,2015-01-14,E07000133,...,4.0,Heating and Natural Ventilation,"Hose Village Hall, 2 Harby Lane",Melton,Rutland and Melton,MELTON MOWBRAY,2015-01-19,NaN,NaN,NaN
95002220202015011914262431900280,NaN,RCCG,Covenant Restoration Assembly,B5 7AY,121382930000,246,G,D1 Non-residential Institutions - Community/Da...,2014-08-29,E08000025,...,4.0,Heating and Natural Ventilation,"RCCG, Covenant Restoration Assembly",Birmingham,"Birmingham, Ladywood",BIRMINGHAM,2015-01-19 14:26:24,NaN,NaN,NaN
96436380962015011914380813200250,NaN,51 Great Ancoats Street,NaN,M4 5AE,515481360000,673,G,B2 to B7 General Industrial and Special Indust...,2014-10-22,E08000003,...,4.0,Heating and Natural Ventilation,51 Great Ancoats Street,Manchester,Manchester Central,MANCHESTER,2015-01-19 14:38:08,NaN,NaN,NaN
98603895032015011915303718000897,NaN,30 High Street,NaN,BA9 9JF,978889090000,143,F,A1/A2 Retail and Financial/Professional services,2015-01-19,E07000189,...,4.0,Heating and Natural Ventilation,30 High Street,South Somerset,Somerton and Frome,WINCANTON,2015-01-19 15:30:37,NaN,NaN,NaN
8777960642015011916584311600090,NaN,Harlow District Council,The Stow,CM20 3AQ,563496610000,158,G,B1 Offices and Workshop businesses,2015-01-09,E07000073,...,4.0,Heating and Natural Ventilation,"Harlow District Council, The Stow",Harlow,Harlow,HARLOW,2015-01-19 16:58:43,NaN,NaN,NaN
98581580202015011918180521000090,NaN,24-30 West Smithfield Street,NaN,EC1A 9DL,425788020000,142,F,B1 Offices and Workshop businesses,2015-01-09,E09000001,...,5.0,Air Conditioning,24-30 West Smithfield Street,City of London,Cities of London and Westminster,LONDON,2015-01-19 18:18:05,NaN,NaN,NaN


In [44]:
# how many are currently non-compliant
len(dedup[dedup['asset-rating-band'].isin(['F', 'G'])]) # 103,935

103935

In [45]:
# how many with no uprn are non-compliant
len(dedup[
        (dedup["uprn"].isna()) &
        (dedup['asset-rating-band'].isin(['F', 'G']))
    ]) # 41985 (out of 103935)

41985

Observations:
- I don't think we can discount the records before 2014. They are expired, so potentially just as bad, there's plenty in the F and G bands.
- More missing UPRNs in the most recent year 40k, then it drops down to median value 20k-30k from 2022 and before
- There's a lot less in F and G in more recent years. 1175 in 2023, 10740 in 2013
- ~40% of non-compliant properties are missing UPRNs (for all dates)

We will require 4 separate files from DMS:
- EPC list
- os, hmlr, voa, ch data match list
- exemptions (could potentially be included in EPC, but best wait till see actual exemptions data)
- epc - os match list (single record for uprn, multiple for address)

Match algorithm:
- call os api for the 40% with no uprn
- v2 could allow LATS to "secure" the match
- v3 could submit this secured match back to EPC.